In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("..")


In [3]:
import pandas as pd
import mediapipe as mp
from src.data import Video
from src.distances import lcss_dist
from src.models.model import kNNClassifier
from src.resources.joints import JOINTS
from src.visualization import plot_3d_landmarks


<img src="https://i.imgur.com/3j8BPdc.png" style="width:800px" >

In [4]:
DATA_PROCESSED_FILEPATH = "data\\processed"

X_train, y_train = [], []

for (dirpath, _, filenames) in os.walk(DATA_PROCESSED_FILEPATH):
    for frame_name in filenames:
        x = pd.read_csv(os.path.join(dirpath, frame_name), index_col=0)
        y = dirpath.split("\\")[-1]
        X_train.append(x)
        y_train.append(y)


In [5]:
x

,0.0,0.033333333,0.066666666,0.1,0.133333333,0.166666666,0.2,0.233333333,0.266666666,0.3,...,13.9,13.933333333,13.966666666,14.0,14.033333333,14.066666666,14.1,14.133333333,14.166666666,14.2
"(left_shoulder,\n left_elbow,\n left_wrist)",134.472415,135.681745,136.865995,138.022451,139.148307,140.240971,141.298448,142.319789,143.305519,144.257823,...,150.486841,149.881567,149.291967,148.722280,148.176066,147.656133,147.164446,146.702102,146.269370,145.865774
"(left_elbow,\n left_shoulder,\n left_hip)",38.538229,42.963269,47.310769,51.574067,55.746771,59.823080,63.798201,67.668862,71.434013,75.095932,...,37.704207,34.060474,30.441868,26.855436,23.307221,19.802212,16.344301,12.936215,9.579445,6.274162
"(left_shoulder,\n left_hip,\n left_knee)",165.409703,159.653698,154.001747,148.465566,143.057229,137.788744,132.671559,127.715947,122.930304,118.320390,...,155.300942,160.850943,166.366417,171.836739,177.252716,182.606692,187.892657,193.106356,198.245392,203.309353
"(nose,\n left_hip,\n left_ankle)",173.710876,172.923511,172.153938,171.403133,170.671845,169.960555,169.269434,168.598355,167.946878,167.314268,...,173.074067,173.679337,174.289446,174.903667,175.521241,176.141422,176.763506,177.386853,178.010909,178.635222
"(left_hip,\n left_knee,\n left_ankle)",167.611785,161.420560,155.349492,149.409820,143.612510,137.967790,132.484725,127.170825,122.031709,117.070739,...,162.577261,169.018359,175.441337,181.835599,188.191651,194.501270,200.757692,206.955771,213.092117,219.165275
"(left_elbow,\n nose,\n right_elbow)",38.125589,37.550290,36.973929,36.397556,35.822497,35.250300,34.682712,34.121642,33.569097,33.027061,...,35.072449,34.374446,33.665153,32.945708,32.217205,31.480710,30.737242,29.987796,29.233352,28.474825
"(left_hip,\n nose,\n right_hip)",10.172427,10.251762,10.329508,10.405600,10.479990,10.552648,10.623560,10.692731,10.760188,10.825995,...,11.331010,11.274511,11.217148,11.158881,11.099686,11.039554,10.978492,10.916522,10.853675,10.789999
"(left_shoulder,\n nose,\n right_shoulder)",37.974179,38.731439,39.478218,40.213617,40.936778,41.646910,42.343319,43.025436,43.692863,44.345404,...,41.097688,40.427681,39.751424,39.069668,38.383152,37.692599,36.998704,36.302124,35.603476,34.903310
orientation,6.427362,6.484728,6.540611,6.594882,6.647417,6.698099,6.746823,6.793508,6.838097,6.880564,...,7.519348,7.401028,7.282929,7.165218,7.048047,6.931555,6.815858,6.701052,6.587204,6.474349


In [6]:
X_test = []

for _ in range(1):
    vid = Video(path=1)
    vid.get_landmarks(model_complexity=2, show=True)
    vid.extract_coordinates()
    vid.extract_angles(JOINTS)

    vid._angles.smooth(inplace=True)
    # vid._angles.scale(inplace=True)

    X_test.append(vid._angles.frame)


webcam: 51it [00:07,  6.46it/s]


In [7]:
knn = kNNClassifier(k=4, epsilon=150)
knn.fit(X=X_train, y=y_train)
knn._predict(X_test)

[{'pullup': 0.5, 'pushup': 0.5}]

In [8]:
import pandas as pd

predictions = knn._predict(X_test)
pd.DataFrame(predictions)

,pullup,pushup
0,0.5,0.5


In [11]:
plot_3d_landmarks(
    landmark_list=vid.landmarks_series[-5],
    connections=mp.solutions.pose.POSE_CONNECTIONS,
)
